### Simple Harmonic Oscillator

This notebook solves a driven simple harmonic oscillator system written in first-order form,
$$
\begin{aligned}
\dot u &= v\\
\dot v &= -100 u + 8\cos(9t),
\end{aligned}
$$
using the `DifferentialEquations.jl` package in Julia.  This package is one of the best collections
of ODE solvers available. This notebook demonstrates how easy it is to use.

The solution vector is 
$$
y = \left(\begin{array}{c} u\\ v\end{array}\right).
$$

In [1]:
using Plots
using DifferentialEquations

#### Function to evaluate the RHS of the ODEs

In [2]:
function deq2!(du, u, p, t)
    du[1] = u[2]
    du[2] = -100.0*u[1] + 8.0*cos(9.0*t) 
end

deq2! (generic function with 1 method)

#### Calculate the exact solutions

These functions calculate the exact solution of $u$ (`fex2!`) and $v$ (`dfex2!`).  `ferr2!` calculates a normed error that can be plotted on log scales.

In [3]:
function fex2!(f, p, t)
    @. f = p[1]*cos(10.0*t) + p[2]*sin(10.0*t) + 8.0/19.0*cos(9.0*t)
end
function dfex2!(df, p, t)
    @. df = -10.0*p[1]*sin(10.0*t) + 10.0*p[2]*cos(10.0*t) - 72.0/19.0*sin(9.0*t)
end
function ferr2!(e, fh, fex)
    epsilon = 1.23453569899e-16
    @. e = abs(fh - fex) + epsilon
end

ferr2! (generic function with 1 method)

#### Set the initial data

In [4]:
p0 = [1.0, 0.0]
u01 = [0.0]
u02 = [0.0]
t0 = [0.0]
fex2!(u01, p0, t0)
dfex2!(u02, p0, t0)
u0 = [u01[1], u02[1]]

2-element Vector{Float64}:
 1.4210526315789473
 0.0

#### Define the ODE problem for the DifferentialEquations.jl package

Notice that we integrate to $t=15$ here.  Using the Midpoint method with a fixed time step, we only went to $t=5$.

In [5]:
tspan = (0.0, 15.0)
prob = ODEProblem(deq2!, u0, tspan)

ODEProblem with uType Vector{Float64} and tType Float64. In-place: true
timespan: (0.0, 15.0)
u0: 2-element Vector{Float64}:
 1.4210526315789473
 0.0

#### Define parameters for the ODE solver

These parameters are used by the ODE integrator.  `alg` defines the method to use, `adapt` is either true or false, or adaptive or fixed time steps, respectively.  Finally, the solution is saved at intervals of `outdt`.  Internally, the algorithm uses smaller steps when necessary to meet the error tolerances set by `reltol` and `abstol`.

In [6]:
#alg = Midpoint()
#alg = ABM54()
alg = Vern9()
#alg = RK4()
#alg = Tsit5()
adapt = true
outdt = 0.05

0.05

#### Call the ODE solver

This is where the solution is calculated.

In [7]:
sol = solve(prob, alg, reltol=1.0e-12, abstol=1.0e-12, adaptive=adapt, saveat=outdt);

#### Plot solutions

The exact solution for $u$ is calculated and compared to the numerical solution.  Using the high-order
methods with adaptive time stepping means that the error is extremely small.  In this case I'm plotting only the exact solution.

In [8]:
tt = sol.t;
ex1 = similar(tt)
fex2!(ex1,p0,tt)
uh = sol[1,:];

In [ ]:
plot(tt,ex1,
    lw=2,
    grid = :off,
    xtickfont=font(12),
    ytickfont=font(12),
    guidefont=font(12),
    linecolor=:blue,
    legend=false)
plot!(tt,uh,label="s16",linecolor=:green,lw=2)

#### Plot the error in the numerical solution

The error in the numerical solution is too small to be seen by eye in this example.  So we plot the difference, and see that it is below our tolerances of $10^{-12}$.

In [ ]:
err1 = similar(uh)
ferr2!(err1,uh,ex1)
plot(tt, err1,label="u_err")